In [18]:
import copy, sys
import random, time, numpy as np

sys.path.append("../../")
from importlib import reload

import inferring.InferringDFA as InferringDFA
import utils.runLearnLib.RunLearnLib as RunLearnLib
import utils.automata.DFA
import utils.advice_systems.SRS as SRS
import utils.display.tables as Tables

reload(InferringDFA)
reload(RunLearnLib)
reload(utils.automata.DFA)
reload(SRS)
reload(Tables)

from inferring.InferringDFA import InferringDFA
from utils.runLearnLib.RunLearnLib import RunLearnLib
from utils.automata.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS
from utils.display.tables import Tables 

In [19]:
def run_learning_process(target, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(target)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [20]:
class Record:

    def __init__(self,
                 d1,
                 d2,
                 conv,
                #  lstar_with_advice,
                 target=None,
                 ttt_eq=None, 
                 ttt_wa_eq=None):
        self.d1 = d1
        self.d2 = d2
        self.conv = conv
        # self.lstar_with_advice = lstar_with_advice
        self.ttt_eq = ttt_eq
        self.ttt_wa_eq = ttt_wa_eq 
        self.target = target

    def print_record(self):
        print(
            f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv}"
        )
    def print_target(self):
        print(self.target.print_complete_description())

In [ ]:
number_of_itreations = 10

#  Fixed random seeds for reproducibility
seeds = [i for i in range(number_of_itreations)]
# The alphabet of the DFA
input_signs = ["a", "b", "c"]
# The bound on the number of states
max_number_of_states = 50
# The list to aggregate the results
results = []

helper = RunLearnLib()

# Build the necessary part of learnLib project.
helper.compileLearnLib()

i = 0
while i < number_of_itreations:
    random.seed(seeds[i])
    i += 1
    print(f"iter nr: {i}")

    # Create two random DFAs and run the learning process to minimize them.
    # Exclude the degenerate cases (DFA of size smaller than 2).
    d1, d2 = DFA(), DFA()
    while d1.Q < 2:
        dfa1 = DFA()
        dfa1.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs,
        )
        d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))
    while d2.Q < 2:
        dfa2 = DFA()
        dfa2.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs,
        )
        d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    # Create a convolution DFA based on d1 and d2
    conv_dfa = DFA()
    conv_dfa.create_convolution(d1, d2)

    # Learn conv_dfa with advice, and store d - learned automaton
    start_t = time.time()
    d, lstar_with_advice, lstar_ex_with_advice = run_learning_process(
        target=conv_dfa, advice_system=SRS(), check_consistency=True)
    end_t = time.time()

    d.type = DFA.CONV_DFA

    results.append(
        Record(
            d1=d1.Q,
            d2=d2.Q,
            conv=d.Q,
            # lstar_with_advice=(lstar_with_advice, lstar_ex_with_advice),
            target=copy.deepcopy(d),
        ))

    results[-1].target.save_complete_description()

    # Learn d automaton without advice using LearnLib implementation of TTT.
    ttt_output = helper.runTTT()
    # Learn d automaton with advice using LearnLib implementation of TTT.
    ttt_wa_output = helper.runTTTWithAdvice()

    results[-1].ttt_eq = ttt_output[1]
    results[-1].ttt_wa_eq = ttt_wa_output[1]


In [14]:
results.sort(key=lambda x: x.conv)

In [15]:
ttt_eqs = [r.ttt_eq for r in results]
ttt_wa_eqs = [r.ttt_wa_eq for r in results]
red = ((np.array(ttt_eqs) - np.array(ttt_wa_eqs)) / np.array(ttt_eqs)) * 100

columns = [('Target language', 'conv(DFA1, DFA2)'), ('Target language', 'DFA1'), ('Target language', 'DFA2'), ('TTT', 'EQ'), ('TTT with advice', 'EQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.conv for r in results],
    columns[1]: [r.d1 for r in results],
    columns[2]: [r.d2 for r in results],
    columns[3]: ttt_eqs,
    columns[4]: ttt_wa_eqs,
    columns[5]: [int(r) for r in red]
}

table_creator = Tables()
df = table_creator.create_basic_table(columns=columns, data=data)
display(df)

Target language             TTT TTT with advice Reduction
     conv(DFA1, DFA2) DFA1 DFA2    EQ              EQ        EQ
0                 598   26   23   583              14       97%
1                 667   29   23   658              12       98%
2                 864   32   27   850              16       98%
3                 864   32   27   850              18       97%
4                 910   35   26   866               2       99%
5                1050   30   35  1004               8       99%
6                1225   35   35  1204              18       98%
7                1488   48   31  1469              18       98%
8                1911   39   49  1887               8       99%
9                2024   44   46  1898               3       99%
10                                                             
11      Max reduction                                       99%
12      Min reduction                                       97%
13  Average reduction                                       98%